# Embeddings Module

### Install necessary modules

In [1]:
%pip install -U -e ..
%pip install -e ..
%pip install pydantic==2.9.2
%pip install nest_asyncio

Obtaining file:///Users/jose/Repos/MINE-DD
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached pydantic-2.11.2-py3-none-any.whl.metadata (64 kB)
  Using cached pydantic_core-2.33.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.8 kB)
Using cached pydantic-2.11.2-py3-none-any.whl (443 kB)
Using cached pydantic_core-2.33.1-cp312-cp312-macosx_11_0_arm64.whl (1.9 MB)
  Building editable for minedd (pyproject.toml) ... done
  Created wheel for minedd: filename=minedd-0.1.0-0.editable-py3-none-any.whl size=13035 sha256=1125c2b5666e36bc4fca458ae8cbabbb1b5ce090ae37ffc4311b4ccb22c9d214
  Stored in directory: /private/var/folders/79/zf67ls7520x9m4mj7nx6q07w0000gp/T/pip-ephem-wheel-cache-evkagyy4/wheels/c7/bf/1e/e7790fb2ba7cdeeb0fd2dc5eb82103f3cd4a31d3c897656e15
Successfully built minedd
  Attempting uninstall: pydantic-core
    

In [2]:
import nest_asyncio
nest_asyncio.apply()

### Load Embeddings object

In [3]:
from minedd.embeddings import Embeddings

embeddings = Embeddings(
    output_embeddings_path="my-embeddings.pkl",
)
embeddings

### Load papers for processing

In [4]:
from pathlib import Path
PAPERS_DIRECTORY = Path.home() / "papers_minedd/"
pdf_file_list = embeddings.prepare_papers(PAPERS_DIRECTORY)
print(len(pdf_file_list))
# Check the first 10 files
pdf_file_list[:10]

11


['Ambient temperature and age-related notified Campylobacter infection in Israel_ A 12-year time series study.pdf',
 'A comparison of weather variables linked to infectious disease patterns using laboratory addresses and patient residence addresses.pdf',
 '.DS_Store',
 '_i_Campylobacter__i_ Monitoring in German Broiler Flocks_ An Explorative Time Series Analysis.pdf',
 'A time series analysis of the relationship of ambient temperature and common bacterial enteric infections in two Canadian provinces.pdf',
 'A Bayesian spatio-temporal framework to identify outbreaks and examine environmental and social risk factors for infectious diseases monitored by routine surveillance.pdf',
 'A time-series study of the association of rainfall_ relative humidity and ambient temperature with hospitalizations for rotavirus and norovirus infection among children in Hong Kong.pdf',
 'Weather Variability and the Incidence of Cryptosporidiosis_ Comparison of Time Series Poisson Regression and SARIMA Models

### Define paperQA settings

In [5]:
from paperqa.settings import Settings, AgentSettings

MODEL = "ollama/llama3.2"
EMBEDDING = "ollama/mxbai-embed-large:latest"

"""Configure settings for paperqa."""
local_llm_config = {
    "model_list": [
        {
            "model_name": MODEL,
            "litellm_params": {
                "model": MODEL,
                "api_base": "http://0.0.0.0:11434",
            },
            "answer": {
                "evidence_k": 40,
                "evidence_detailed_citations": True,
                "evidence_summary_length": "about 100 words",
                "answer_max_sources": 10,
                "answer_length": "about 600 words, but can be longer",
                "max_concurrent_requests": 10,
                "answer_filter_extra_background": False
            },
            "parsing": {
                "use_doc_details": True
            }
        }
    ]
}

settings = Settings(
    llm=MODEL,
    llm_config=local_llm_config,
    summary_llm=MODEL,
    summary_llm_config=local_llm_config,
    paper_directory=str(PAPERS_DIRECTORY),
    embedding=EMBEDDING,
    agent=AgentSettings(
        agent_llm=MODEL,
        agent_llm_config=local_llm_config,
        return_paper_metadata=True
    )
)

### Create Embeddings and save in PKL

In [6]:
embeddings.process_papers(settings, PAPERS_DIRECTORY, pdf_file_list)
print("Embeddings created and saved to my-embeddings.pkl")

Creating new Docs object.


  0%|          | 0/11 [00:00<?, ?it/s]/opt/miniconda3/envs/poopqa/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to parse all of title, DOI, and authors from the ParsingSettings.structured_citation_prompt's response {"title": None, "authors": [], "doi": None}, consider using a manifest file or specifying a different citation prompt.
  9%|▉         | 1/11 [00:53<08:51, 53.14s/it]

Correctly loaded Ambient temperature and age-related notified Campylobacter infection in Israel_ A 12-year time series study.pdf


Failed to parse all of title, DOI, and authors from the ParsingSettings.structured_citation_prompt's response {key: value for key, value in [('title', title), ('authors', authors), ('doi', doi)] if value is not None}, consider using a manifest file or specifying a different citation prompt.
 18%|█▊        | 2/11 [01:16<05:18, 35.36s/it]

Correctly loaded A comparison of weather variables linked to infectious disease patterns using laboratory addresses and patient residence addresses.pdf


...

Correctly loaded _i_Campylobacter__i_epidemiology_ a descriptive study reviewing 1 million cases in England and Wales between 1989 and 2011.pdf


Failed to parse all of title, DOI, and authors from the ParsingSettings.structured_citation_prompt's response {4,9}, consider using a manifest file or specifying a different citation prompt.
 91%|█████████ | 10/11 [04:43<00:28, 28.71s/it]

Correctly loaded Weather and notified Campylobacter infections in temperate and sub-tropical regions of Australia_ An ecological study.pdf


SEMANTIC_SCHOLAR_API_KEY environment variable not set. Semantic Scholar API rate limits may apply.
100%|██████████| 11/11 [05:02<00:00, 27.53s/it]

Correctly loaded Are hospitalizations for rotavirus gastroenteritis associated with meteorologic factors_.pdf
Docs object saved to my-embeddings.pkl
Embeddings created and saved to my-embeddings.pkl


### Inspect Documents Object

In [7]:
def get_doc_details(doc_detail, verbose=False):
    def get_safe_key(doc_detail, key):
        try:
            return getattr(doc_detail, key)
        except AttributeError:
            return None

    details_dict = {
        "key": get_safe_key(doc_detail, "key"),
        "docname":get_safe_key(doc_detail, "docname"),
        "title": get_safe_key(doc_detail, "title"),
        "authors": get_safe_key(doc_detail, "authors"),
        "year": get_safe_key(doc_detail, "year"),
        "journal": get_safe_key(doc_detail, "journal"),
        "volume": get_safe_key(doc_detail, "volume"),
        "pages": get_safe_key(doc_detail, "pages"),
        "doi": get_safe_key(doc_detail, "doi"),
        "url": get_safe_key(doc_detail, "url"),
    }

    # Print for informative purposes
    if verbose:
        for k, v in details_dict.items():
            print(f"{k}: {v}")

    return details_dict

for doc_key in embeddings.docs.docs.keys():
    doc_detail = embeddings.docs.docs.get(doc_key)
    print(doc_detail)
    get_doc_details(doc_detail, verbose=True)
    print("-" * 80)

docname='Ambient temperature and age-related notified Campylobacter infection in Israel_ A 12-year time series study.pdf' citation='Alina Rosenberger, Miriam Weinberger, Shlomit Pazd, Lea Valinskye, Vered Agmon, Chava Peretz. "Ambient Temperature and Age-Related Notified Campylobacter Infection in TIsrael: A 12-Year Time Series Study." Environmental Research 164 (2018): 539–545.\n\nWeinberger, Miriam. "Campylobacter Infections: An Overview of a Public Health Problem." Journal of Food Protection 76.5 (2013): 751-758.\n\nEuropean Food Safety Authority. "The European Food Safety Authority\'s Scientific Opinion on the Risk Profile of Campylobacter in Relation to Human Health and the Environment." EFSA Journal 15.10 (2017): e04731.\n\nInstitute of Environmental Science and Research Ltd. "Campylobacter Infection Control in New Zealand." \n\nNote: Since no publication date is provided for the Institute of Environmental Science and Research Ltd\'s report, I could not include it in the citation